# Labelling of pole-like objects
E.g. trees, street lights, traffic signs.

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import src.fusion as fusion
import src.region_growing as growing
import src.utils.las_utils as las_utils
import src.utils.ahn_utils as ahn_utils
import src.utils.clip_utils as clip_utils
from src.utils.labels import Labels
from src.utils.interpolation import FastGridInterpolator

## Set-up

In [ ]:
# Select the point cloud file to process.
tilecode = '2386_9702'

# The corresponding BGT data file.
bgt_data_file = '../datasets/bgt/custom_points_' + tilecode + '.csv'

# We need elevation data to determine object height above ground.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(data_folder=ahn_data_folder)

In [ ]:
# Load a (partially labelled) LAS file, e.g. from the previous step.
las_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
las = las_utils.read_las(las_file)

labels = las.label
points = np.vstack((las.x, las.y, las.z)).T

## Match point objects with clusters

First we try to match all objects in the BGT data with pole-like objects in the point cloud. We look for clusters of points with specific characteristics, such as a minimum height, density, and a maximum radius.

We start with trees, since the location data for these is most precise (in Amsterdam).

In [ ]:
# All possible parameters:
# params = {'search_pad': 1.5,   # Specify the padding (in m) around the BGT object in which to search for a match in the point cloud.
#           'max_dist': 1.2,     # Maximum distance (in m) between the expected location and the location of a potential match.
#           'voxel_res': 0.2,    # Resolution of voxels used when searching for a match.
#           'seed_height': 1.75, # Height above ground at which to determine object dimensions.
#           'min_height': 2.,    # Minimum hieght for an object to be considered a match.
#           'max_r': 0.5,        # Maximum radius for a pole-like object to be considered a match.
#           'min_points': 500,   # Minimum number of points for a cluster to be considered.
#           'z_min': 0.2,        # Height above ground, above which to search for objects.
#           'z_max': 2.7,        # Height above ground, below which to search for objects.
#           'r_mult': 1.5,       # Multiplier for radius when performing the initial (cylinder-based) labelling.
#           'label_height': 4.}  # Maximum height for initial (cylinder-based) labelling.

# Object type-specific parameters.
tree_params = {'seed_height': 1.75, 'min_points': 500, 'max_r': 0.5, 'label_height': 5.}
light_params = {'seed_height': 2.25, 'min_points': 400, 'max_r': 0.2, 'label_height': 5.}
sign_params = {'seed_height': 1.75, 'min_points': 200, 'max_r': 0.2, 'min_height': 1.2, 'z_max': 2., 'label_height': 3.}

# Create fusers for BGT point data.
tree_fuser = fusion.BGTPointFuser(Labels.TREE, bgt_type='boom', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=tree_params)
light_fuser = fusion.BGTPointFuser(Labels.STREET_LIGHT, bgt_type='lichtmast', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=light_params)
sign_fuser = fusion.BGTPointFuser(Labels.TRAFFIC_SIGN, bgt_type='verkeersbord', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=sign_params)

In [ ]:
# Process the point cloud.
mask = (labels != Labels.GROUND) & (labels != Labels.BUILDING) & (labels != Labels.CAR)

start = time.time()
tree_mask = tree_fuser.get_label_mask(points, labels, mask, tilecode)
labels[tree_mask] = tree_fuser.get_label()
mask[tree_mask] = False
end = time.time()
print(f'Trees fused in {end-start:.2f} seconds.\n')

start = time.time()
light_mask = light_fuser.get_label_mask(points, labels, mask, tilecode)
labels[light_mask] = light_fuser.get_label()
mask[light_mask] = False
end = time.time()
print(f'Street lights fused in {end-start:.2f} seconds.\n')

start = time.time()
sign_mask = sign_fuser.get_label_mask(points, labels, mask, tilecode)
labels[sign_mask] = sign_fuser.get_label()
mask[sign_mask] = False
end = time.time()
print(f'Traffic signs fused in {end-start:.2f} seconds.')

## Extend point objects using LCC region growing

In the initial labelling phase we only labelled a cylinder-based region of the point cloud at the object's location.

We now refine the initial label by adding connected components that overlap with the initial label. This is done in two parts: close to the ground we want to be more careful, since there might be more clutter; while higher above ground we want to label the complete object without missing any part. Therefore, we use different region growing parameters for the top and bottom part of the object.

For more details on region growing, see notebook [5. Region growing](5.%20Region%20growing.ipynb)

This time we process the trees last, since these are harder to get right.

In [ ]:
# All possible parameters:
# params = {'plane_height': REQUIRED,  # Height above ground at which to start/stop (for top/bottom) growing.
#           'octree_level': 9,         # Octree level for LCC method, higher means more fine-grained.
#           'min_comp_size': 100,      # Minimum number of points for a component to be considered.
#           'threshold': 0.5}          # Minimum fraction of points in a component that are already labelled initially for the component to be added.

# Parameter settings for top and bottom growing.
light_top_params = {'plane_height': 3.25, 'threshold': 0.05}
light_bottom_params = {'plane_height': 1.5, 'octree_level': 10, 'threshold': 0.5}

sign_top_params = {'plane_height': 1.5, 'threshold': 0.05}
sign_bottom_params = {'plane_height': 1.5, 'octree_level': 10, 'threshold': 0.5}

tree_top_params = {'plane_height': 1.75, 'octree_level': 9, 'threshold': 0.01}
tree_bottom_params = {'plane_height': 1.75, 'octree_level': 10, 'threshold': 0.8}

# Create the region growing objects.
light_lcc = growing.TopBottomLCC(Labels.STREET_LIGHT, ahn_reader, 
                                 top_params=light_top_params, 
                                 bottom_params=light_bottom_params)
sign_lcc = growing.TopBottomLCC(Labels.TRAFFIC_SIGN, ahn_reader, 
                                top_params=sign_top_params, 
                                bottom_params=sign_bottom_params)
tree_lcc = growing.TopBottomLCC(Labels.TREE, ahn_reader, 
                                top_params=tree_top_params, 
                                bottom_params=tree_bottom_params)

In [ ]:
# Process the point cloud.
start = time.time()
light_lcc_mask = light_lcc.get_label_mask(points, labels, mask, tilecode)
labels[light_lcc_mask] = light_lcc.get_label()
mask[light_lcc_mask] = False
end = time.time()
print(f'Street lights grown in {end-start:.2f} seconds.\n')

start = time.time()
sign_lcc_mask = sign_lcc.get_label_mask(points, labels, mask, tilecode)
labels[sign_lcc_mask] = sign_lcc.get_label()
mask[sign_lcc_mask] = False
end = time.time()
print(f'Traffic signs grown in {end-start:.2f} seconds.')

start = time.time()
tree_lcc_mask = tree_lcc.get_label_mask(points, labels, mask, tilecode)
labels[tree_lcc_mask] = tree_lcc.get_label()
mask[tree_lcc_mask] = False
end = time.time()
print(f'Trees grown in {end-start:.2f} seconds.')

### Save LAS file

In [ ]:
# Save the result (this will overwrite the LAS file.)
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
las_utils.label_and_save_las(las, labels, out_file)

## Visualize results

In [ ]:
# Visualize the resulting match.
((x_min, y_max), (x_max, y_min)) = las_utils.get_bbox_from_tile_code(tilecode)

ahn_tile = ahn_reader.filter_tile(tilecode)
fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'],
                              ahn_tile['ground_surface'])
points_z = fast_z(points[:, 0:2])

plane_mask = (points[:, 2] >= points_z + 1.7) & (points[:, 2] <= points_z + 1.8)
label_set = np.unique(labels[plane_mask])

fig, ax = plt.subplots(1, constrained_layout=True)

for label in label_set:
    label_mask = plane_mask & (labels == label)
    scatter = ax.scatter(points[label_mask, 0], points[label_mask, 1], marker='.', label=Labels.get_str(label))

box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='grey', fill=False)

ax.add_patch(box)
ax.set_title(tilecode)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.axis('equal')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()